이전에 split했었던 문서에 embed 작업을 해보자.

그 전에 먼저 embedding model에 대해 살펴보자.

OpenAI가 가지고 있고, LangChain이 지원하는 거다.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

embedder.embed_query("Hi")

```python
embedder.embed_query("Hi")
```
위 코드를 실행하면 embed 작업을 수행한다.

그럼 'Hi'를 표현하는 벡터를 얻게 될 것이다.

차원이 몇 개 인지 확인해 보자.

In [2]:
vector = embedder.embed_query("Hi")
len(vector)

1536

한 개의 단어 Hi를 위한 vector가 1536개 이다. (차원)

이번에는 문서들을 embedding 해보자.  
이것들은 string의 배열로 되어있어야 한다.


In [4]:
# 이것들은 string의 배열로 되어있어야 한다.
vector = embedder.embed_documents([
    'hi',
    'how',
    'are',
    'you longer sentence because',
])

print(len(vector), len(vector[0]))

4 1536


vector에 입력해 준 4개의 각 문서에 하나씩 할당된 것이다.

또 이들 각각은 같은 차원을 갖고 있다.

즉, 위에서 우리는 4개의 벡터가 있고, 각 벡터는 1536개의 차원을 갖고 있다.

이제 실제로 우리 문서를 embed 해보자.

위에서처럼 직접적으로 하진 않을 것이다. 

왜냐하면 문서 embed를 반복해서 수행하고 싶지 않기 때문이다.

대신, 우리는 그 embed들을 저장해 줄거다.

이 방법으로 코스트를 줄일 수 있다.

그리고 랭체인은 임베딩한 것들을 캣이하는 기능을 제공하고 있다.

---  



이제 vector store에 대해 알아보자.

vector store는 일종의 데이터베이스라고 생각할 수 있다.

벡터 공간에서 검색을 할 수 있게 해준다.

그러니까, 우리가 벡터들을 만들고나서 그것들을 캐시해주고 vector store에 그 벡터들을 넣어주면,

우리가 검색을 할 수 있다.

관련있는 문서들만 찾아낼 수 있게 되는 것!



In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(docs, embeddings)



In [7]:
result = vectorstore.similarity_search("where does winston live")
result
# len(result)

[Document(page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never\nbeen inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on\nofficial business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors,\nand hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced\nguards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was\nadvisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the\nMinistry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no\nfood in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast.\nHe took down from the shelf a bottle of colourl

우리는 모든 문서들을 벡터로 바꾸었고, 그 벡터들을 넣은 vector store를 활용해서 검색을 할 수 있게 되었다.

이제 이 query와 비슷한 문서를 찾을 수 있게 된 것이다.

이제 결과를 살펴보자.

그리고 캐시..

캐시 폴더를 만들고 cached embeddings을 생성해보자.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# 캐시 경로
cache_dir = LocalFileStore('./.cache/')



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# .from_bytes_store 메서드는 임베딩 작업을 위해 필요한 embedder의 입력을 요구한다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir,
)

vectorstore = Chroma.from_documents(docs, embeddings)


